In [1]:
'''
    Script for checking if the Inference computations run properly for the trained graph.
'''

from Summary_Generator.Tensorflow_Graph import order_planner_without_copynet
from Summary_Generator.Text_Preprocessing_Helpers.pickling_tools import *
from Summary_Generator.Tensorflow_Graph.utils import *
from Summary_Generator.Model import *
import numpy as np
import tensorflow as tf
#from tensorflow.contrib.tensorboard.plugins import projector
import tf_slim as slim

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# define the class for the Model
class Model:
    '''
        Model for training and Inferring on the task
        This is a simplified implementation of the Tensorflow's estimator module
    '''
    # Helper methods for initialization:
    def __setup_optimizer(self):
        # setup the optimizer with the graph
        with self.graph.as_default():
            # define the optimizer for this task:
        	with tf.variable_scope("Trainer"):
        	    # define the train_step for this:
        	    self.train_step = self.optimizer.minimize(self.loss)

        	with tf.variable_scope("Init"):
        		self.init = tf.global_variables_initializer()

    def __setup_graph(self):
        # print all the trainable variables for this graph
        with tf.Session(graph=self.graph) as sess:
            # initialize the session to generate the visualization file
            sess.run(self.init)

            tvars = tf.trainable_variables()
            tvars_vals = sess.run(tvars)

            print("\n\nAll the trainable variables in the Graph: ")
            for var, val in zip(tvars, tvars_vals):
                print(var.name)
            print("\n\n")

    def __get_tensorboard_writer(self, path):
        tensorboard_writer = tf.summary.FileWriter(path, graph=self.graph, filename_suffix=".bot")

        # set the projector's configuration to add the embedding summary also:
        conf = projector.ProjectorConfig()
        embedding_field = conf.embeddings.add()
        embedding_content_label = conf.embeddings.add()

        # set the tensors to these embedding matrices
        embedding_field.tensor_name = self.field_embedding_matrix.name
        embedding_content_label.tensor_name = self.content_label_embedding_matrix.name

        # add the metadata paths to these embedding_summaries:
        embedding_field.metadata_path = os.path.join("..", "Metadata/fields.vocab")
        embedding_content_label.metadata_path = os.path.join("..", "Metadata/content_labels.vocab")

        # save the configuration file for this
        projector.visualize_embeddings(tensorboard_writer, conf)

        # return the so created tensorboard_writer
        return tensorboard_writer

    # define the constructor of the graph
    def __init__(self, graph, interface_dict, optimizer, field_vocabulary, content_label_vocabulary):
        '''
            constructor of the class

            graph = The tensorflow graph of the network
            optimizer = The tensorflow optimizer object that is to be used for optimization
        '''
        self.graph = graph
        self.optimizer = optimizer

        # extract the parameters from the interface_dict:
        self.loss = interface_dict["loss"]
        self.all_summaries = interface_dict["summary"]
        self.outputs = interface_dict["training_output"]
        self.inference = interface_dict["inference"]

        self.inp_field_encodings = interface_dict["input"]["field_encodings"]
        self.inp_content_encodings = interface_dict["input"]["content_encodings"]
        self.inp_label_encodings = interface_dict["input"]["label_encodings"]
        self.inp_sequence_lengths = interface_dict["input"]["input_sequence_lengths"]
        self.lab_sequence_lengths = interface_dict["input"]["label_sequence_lengths"]

        self.field_embedding_matrix = interface_dict["field_embeddings"]
        self.content_label_embedding_matrix = interface_dict["content_label_embeddings"]

        # setup the vocabularies for the Model:
        self.field_vocabulary = field_vocabulary
        self.content_label_vocabulary = content_label_vocabulary

        self.__setup_optimizer()
        self.__setup_graph()

    # function for training the graph
    def train(self, X, Y, batch_size, no_of_epochs, checkpoint_factor, model_save_path, model_name):
        '''
            The training_function for the model.
        '''
        # Setup a tensorboard_writer:
        tensorboard_writer = self.__get_tensorboard_writer(model_save_path)

        ''' Start the actual Training loop: '''
        print("\n\nStarting the Training ... ")
        with tf.Session(graph=self.graph) as sess:
            # create a saver object:
            saver = tf.train.Saver(max_to_keep=3)

            # If old weights found, restart the training from there:
            checkpoint_file = os.path.join(model_save_path, "checkpoint")
            if(os.path.isfile(checkpoint_file)):
                # load the saved weights:
                saver.restore(sess, tf.train.latest_checkpoint(model_save_path))

                # load the global_step value from the checkpoint file
                with open(checkpoint_file, 'r') as checkpoint:
                    path = checkpoint.readline().strip()
                    global_step = int((path.split(':')[1]).split('-')[1][:-1])

            # otherwise initialize all the weights
            else:
                sess.run(self.init)

                # set the global_step to 0
                global_step = 0

            print("global_step: ", global_step)

            # run a loop for no_of_epochs iterations:
            for epoch in range(no_of_epochs):
                print("------------------------------------------------------------------------------------------------------------")
                print("current_epoch: ", (epoch + 1))

                # perform random shuffling on the training data:
                X, Y = synch_random_shuffle_non_np(zip(X[0], X[1]), Y)

                # unzip the shuffled X:
                X = zip(*X)

                # setup the data for training:
                # obtain the padded training data:
                train_X_field = X[0]; train_X_content = X[1]
                train_Y = Y; no_of_total_examples = len(train_X_field)

                # print len(train_X_field), len(train_X_content), len(train_Y)
                assert len(train_X_field) == len(train_X_content) and len(train_X_field) == len(train_Y), "input data lengths incompatible"

                # Iterate over the batches of the given train data:
                for batch_no in range(int(np.ceil(float(no_of_total_examples) / batch_size))):
                    # obtain the current batch of data:
                    start = (batch_no * batch_size); end = start + batch_size
                    batch_inp_field = train_X_field[start: end]
                    batch_inp_conte = train_X_content[start: end]
                    batch_inp_label = train_Y[start: end]
                    # pad the current batch of data:
                    inp_field = pad_sequences(batch_inp_field)
                    inp_conte = pad_sequences(batch_inp_conte)
                    inp_label = pad_sequences(batch_inp_label)
                    # extract the sequence lengths of examples in this batch
                    inp_lengths = get_lengths(batch_inp_field)
                    lab_lengths = get_lengths(batch_inp_label)


                    # execute the cost and the train_step
                    _, cost = sess.run([self.train_step, self.loss], feed_dict = {
                        self.inp_field_encodings: inp_field,
                        self.inp_content_encodings: inp_conte,
                        self.inp_label_encodings: inp_label,
                        self.inp_sequence_lengths: inp_lengths,
                        self.lab_sequence_lengths: lab_lengths
                    })
                    print("Range: ", "[", start, "-", (start + len(inp_field)), "]", " Cost: ", cost)
                    global_step += 1

                if((epoch + 1) % checkpoint_factor == 0 or epoch == 0):
                    # generate the summary for this batch:
                    sums, predicts = sess.run([self.all_summaries, self.outputs], feed_dict = {
                        self.inp_field_encodings: inp_field,
                        self.inp_content_encodings: inp_conte,
                        self.inp_label_encodings: inp_label,
                        self.inp_sequence_lengths: inp_lengths,
                        self.lab_sequence_lengths: lab_lengths
                    })

                    # save this generated summary to the summary file
                    tensorboard_writer.add_summary(sums, global_step=global_step)

                    # also save the model
                    saver.save(sess, os.path.join(model_save_path, model_name), global_step=global_step)

                    # print a random sample from this batch:
                    random_index = np.random.randint(len(inp_field))

                    random_label_sample = inp_label[random_index]
                    random_predicts_sample = np.argmax(predicts, axis = -1)[random_index]

                    # print the extracted sample in meaningful format
                    print("\nOriginal Summary: ")
                    print([self.content_label_vocabulary[label] for label in random_label_sample])

                    print("\nPredicted Summary: ")
                    print([self.content_label_vocabulary[label] for label in random_predicts_sample])

                print("------------------------------------------------------------------------------------------------------------")
        print("Training complete ...\n\n")

In [3]:
from Summary_Generator.Tensorflow_Graph import order_planner_with_copynet
from Summary_Generator.Text_Preprocessing_Helpers.pickling_tools import *
from Summary_Generator.Tensorflow_Graph.utils import *
from Summary_Generator.Model import *
import numpy as np
import tensorflow as tf

In [4]:

# random_seed value for consistent debuggable behaviour
seed_value = 3

np.random.seed(seed_value) # set this seed for a device independant consistent behaviour

''' Set the constants for the script '''
# various paths of the files
data_path = "../Data" # the data path

data_files_paths = {
    "table_content": os.path.join(data_path, "train.box"),
    "nb_sentences" : os.path.join(data_path, "train.nb"),
    "train_sentences": os.path.join(data_path, "train.sent")
}

base_model_path = "Models"
plug_and_play_data_file = os.path.join(data_path, "plug_and_play.pickle")

In [5]:
''' Name of the model:  '''
# This can be changed to create new models in the directory
model_name = "Model_2(with_copy_net)"

'''
	=========================================================================================================
	|| All Tweakable hyper-parameters
	=========================================================================================================
'''
# constants for this script
train_percentage = 90
batch_size = 3
checkpoint_factor = 10
learning_rate = 3e-4 # for learning rate -> https://twitter.com/karpathy/status/801621764144971776?lang=en
# I know the tweet was a joke, but I have noticed that this learning rate works quite well.

# using a different learning_rate
learning_rate = 3e-6

# Embeddings size:
field_embedding_size = 100
content_label_embedding_size = 400 # This is a much bigger vocabulary compared to the field_name's vocabulary

# LSTM hidden state sizes
lstm_cell_state_size = hidden_state_size = 500 # they are same (for now)
'''
	=========================================================================================================
'''

'\n\t=========================================================================================================\n'

In [6]:
''' Extract and setup the data '''
# Obtain the data:
data = unPickleIt(plug_and_play_data_file)

field_encodings = data['field_encodings']
field_dict = data['field_dict']

content_encodings = data['content_encodings']

label_encodings = data['label_encodings']
content_label_dict = data['content_union_label_dict']
rev_content_label_dict = data['rev_content_union_label_dict']

# vocabulary sizes
field_vocab_size = data['field_vocab_size']
content_label_vocab_size = data['content_label_vocab_size']

In [7]:
# function to perform synchronous random shuffling of the training data
def synch_random_shuffle_non_np(X, Y):
    '''
        ** This function takes in the parameters that are non numpy compliant dtypes such as list, tuple, etc.
        Although this function works on numpy arrays as well, this is not as performant enough
        @param
        X, Y => The data to be shuffled
        @return => The shuffled data
    '''
    combinedNew = list(zip(X, Y))

    # shuffle the combined list in place
    np.random.shuffle(combinedNew)

    # extract the data back from the combined list
    X, Y = zip(*combinedNew)

    # return the shuffled data:
    return X, Y

X, Y = synch_random_shuffle_non_np(zip(field_encodings, content_encodings), label_encodings)

train_X, train_Y, dev_X, dev_Y = split_train_dev(X, Y, train_percentage)
train_X_field, train_X_content = zip(*train_X)
train_X_field = list(train_X_field); train_X_content = list(train_X_content)

# Free up the resources by deleting non required stuff
del X, Y, field_encodings, content_encodings, train_X

In [8]:
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

In [9]:
''' Obtain the TensorFlow graph of the order_planner_without_copynet Network'''
# just execute the get_computation_graph function here:
graph, interface_dict = order_planner_with_copynet.get_computation_graph (
	seed_value = seed_value,

	# vocabulary sizes
	field_vocab_size = field_vocab_size,
	content_label_vocab_size = content_label_vocab_size,

	# Embeddings size:
	field_embedding_size = field_embedding_size,
	content_label_embedding_size = content_label_embedding_size,

	# LSTM hidden state sizes
	lstm_cell_state_size = lstm_cell_state_size,
	hidden_state_size = hidden_state_size, # they are same (for now)
	rev_content_label_dict = rev_content_label_dict
)



Building the graph ... 

step 1: Creating input placeholders for the computations ...
	placeholder for the field_encodings:  Tensor("Input_Data/input_field_encodings:0", shape=(?, ?), dtype=int32)
	placeholder for the content_encodings:  Tensor("Input_Data/input_content_encodings:0", shape=(?, ?), dtype=int32)
	placeholder for the label_encodings:  Tensor("Input_Data/input_label_encodings:0", shape=(?, ?), dtype=int32)
	placeholder for the input_sequence_lengths:  Tensor("Input_Data/input_sequence_lengths:0", shape=(?,), dtype=int32)
	placeholder for the label_sequence_lengths:  Tensor("Input_Data/decoder_sequence_lengths:0", shape=(?,), dtype=int32)

step 2: Creating Embeddings Mechanism for the input and the output words ...
	Embedded_Input_Tensors:  Tensor("Input_Embedder/field_embedder/Identity:0", shape=(?, ?, 100), dtype=float32) Tensor("Input_Embedder/content_embedder/Identity:0", shape=(?, ?, 400), dtype=float32)
	Embedded_Label_Tensors:  Tensor("Label_Embedder/label_embedder

C:\Users\anusa\OneDrive\Documents\Data Science\natural-language-summary-generation-from-structured-data-Animesh\TensorFlow_implementation\Summary_Generator\Tensorflow_Graph\order_planner_with_copynet.py:109: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.nn.rnn_cell.LSTMCell(lstm_cell_state_size), # let all parameters to be default
C:\Users\anusa\OneDrive\Documents\Data Science\natural-language-summary-generation-from-structured-data-Animesh\TensorFlow_implementation\Summary_Generator\Tensorflow_Graph\order_planner_with_copynet.py:294: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  decoder_cell = tf.nn.rnn_cell.LSTMCell(lstm_cell_state_size)


	Encoded_vectors_bank for attention mechanism:  Tensor("Encoder/rnn/transpose_1:0", shape=(?, ?, 500), dtype=float32)
	Final_state obtained from the last step of encoder:  LSTMStateTuple(c=<tf.Tensor 'Encoder/rnn/while/Exit_3:0' shape=(?, 500) dtype=float32>, h=<tf.Tensor 'Encoder/rnn/while/Exit_4:0' shape=(?, 500) dtype=float32>)

step 4: defining the Attention Mechanism for the Model (The Dispatcher Module) ...
**step 4.1: defining the content based attention
**step 4.2: defining the link based attention
	The Link Matrix used for this attention:  <tf.Variable 'Link_Based_Attention/trainable_weights/Link_Attention_Matrix:0' shape=(106, 106) dtype=float32_ref>
**step 4.3: defining the hybrid attention

step 5: creating the decoder RNN to obtain the generated summary for the structured data (The Decoder Module) ...

step 6: defining the training computations ...
Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().
ERROR:te

Instructions for updating:
Use fn_output_signature instead
	Final Output_Tensor obtained from the decoder:  Tensor("Training_computations/transpose:0", shape=(?, ?, 421), dtype=float32)

step 7: defining the cost function for optimization ...
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.


step 8: defining the computations for the inference mode ...
ERROR:tensorflow:Error: Input value Tensor("inference_computations/Decoder/rnn/while/while/ScatterAdd:0", shape=(421,), dtype=float32_ref) has dtype <dtype: 'float32_ref'>, but expected dtype <dtype: 'float32'>.  This leads to undefined behavior and will be an error in future versions of TensorFlow.  Traceback:
  File "C:\Users\anusa\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\anusa\AppData\Loc

	Inference outputs:  Tensor("inference_computations/transpose:0", shape=(?, ?, 421), dtype=float32)

step _ : setting up the errands for TensorFlow ...




In [10]:
base_model_path

'Models'

In [11]:
''' Start the Training of the data '''
# Create the model and start the training on it
model_path = os.path.join(base_model_path, model_name)
tf.get_variable_scope().reuse_variables()
model = Model(graph, interface_dict, tf.train.AdamOptimizer(learning_rate), field_dict, content_label_dict)
model.train((train_X_field, train_X_content), train_Y, batch_size, 100, checkpoint_factor, model_path, model_name)



All the trainable variables in the Graph: 
Unified_Vocabulary_Matrix/content_label_embedding_matrix:0
Input_Embedder/field_embedding_matrix:0
Encoder/rnn/lstm_cell/kernel:0
Encoder/rnn/lstm_cell/bias:0
Content_Based_Attention/trainable_weights/field_attention_weights:0
Content_Based_Attention/trainable_weights/field_attention_biases:0
Content_Based_Attention/trainable_weights/content_attention_weights:0
Content_Based_Attention/trainable_weights/content_attention_biases:0
Link_Based_Attention/trainable_weights/Link_Attention_Matrix:0
Hybrid_attention/trainable_weights/zt_gate_parameter_vector:0
Decoder/trainable_weights/output_projector_matrix:0
Decoder/trainable_weights/output_projector_biases:0
Decoder/trainable_weights/x_t_gate_matrix:0
Decoder/trainable_weights/x_t_gate_biases:0
Decoder/trainable_weights/copy_matrix:0
Decoder/rnn/lstm_cell/kernel:0
Decoder/rnn/lstm_cell/bias:0





Starting the Training ... 
INFO:tensorflow:Restoring parameters from Models\Model_2(with_copy_net)\M

Range:  [ 3 - 6 ]  Cost:  642.1079
name: "Trainer/Adam"
op: "NoOp"
input: "^Trainer/Adam/update_Unified_Vocabulary_Matrix/content_label_embedding_matrix/group_deps"
input: "^Trainer/Adam/update_Input_Embedder/field_embedding_matrix/group_deps"
input: "^Trainer/Adam/update_Encoder/rnn/lstm_cell/kernel/ApplyAdam"
input: "^Trainer/Adam/update_Encoder/rnn/lstm_cell/bias/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/field_attention_weights/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/field_attention_biases/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/content_attention_weights/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/content_attention_biases/ApplyAdam"
input: "^Trainer/Adam/update_Link_Based_Attention/trainable_weights/Link_Attention_Matrix/group_deps"
input: "^Trainer/Adam/update_Hybrid_attention/trainable_weights/zt_gate_parameter_vector/App

Range:  [ 0 - 3 ]  Cost:  610.92914
name: "Trainer/Adam"
op: "NoOp"
input: "^Trainer/Adam/update_Unified_Vocabulary_Matrix/content_label_embedding_matrix/group_deps"
input: "^Trainer/Adam/update_Input_Embedder/field_embedding_matrix/group_deps"
input: "^Trainer/Adam/update_Encoder/rnn/lstm_cell/kernel/ApplyAdam"
input: "^Trainer/Adam/update_Encoder/rnn/lstm_cell/bias/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/field_attention_weights/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/field_attention_biases/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/content_attention_weights/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/content_attention_biases/ApplyAdam"
input: "^Trainer/Adam/update_Link_Based_Attention/trainable_weights/Link_Attention_Matrix/group_deps"
input: "^Trainer/Adam/update_Hybrid_attention/trainable_weights/zt_gate_parameter_vector/Ap

Range:  [ 6 - 9 ]  Cost:  19.071577
------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------
current_epoch:  3
name: "Trainer/Adam"
op: "NoOp"
input: "^Trainer/Adam/update_Unified_Vocabulary_Matrix/content_label_embedding_matrix/group_deps"
input: "^Trainer/Adam/update_Input_Embedder/field_embedding_matrix/group_deps"
input: "^Trainer/Adam/update_Encoder/rnn/lstm_cell/kernel/ApplyAdam"
input: "^Trainer/Adam/update_Encoder/rnn/lstm_cell/bias/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/field_attention_weights/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/field_attention_biases/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/content_attention_weights/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_w

Range:  [ 3 - 6 ]  Cost:  39.329098
name: "Trainer/Adam"
op: "NoOp"
input: "^Trainer/Adam/update_Unified_Vocabulary_Matrix/content_label_embedding_matrix/group_deps"
input: "^Trainer/Adam/update_Input_Embedder/field_embedding_matrix/group_deps"
input: "^Trainer/Adam/update_Encoder/rnn/lstm_cell/kernel/ApplyAdam"
input: "^Trainer/Adam/update_Encoder/rnn/lstm_cell/bias/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/field_attention_weights/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/field_attention_biases/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/content_attention_weights/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/content_attention_biases/ApplyAdam"
input: "^Trainer/Adam/update_Link_Based_Attention/trainable_weights/Link_Attention_Matrix/group_deps"
input: "^Trainer/Adam/update_Hybrid_attention/trainable_weights/zt_gate_parameter_vector/Ap

...

Range:  [ 6 - 9 ]  Cost:  9.836422
------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------
current_epoch:  98
name: "Trainer/Adam"
op: "NoOp"
input: "^Trainer/Adam/update_Unified_Vocabulary_Matrix/content_label_embedding_matrix/group_deps"
input: "^Trainer/Adam/update_Input_Embedder/field_embedding_matrix/group_deps"
input: "^Trainer/Adam/update_Encoder/rnn/lstm_cell/kernel/ApplyAdam"
input: "^Trainer/Adam/update_Encoder/rnn/lstm_cell/bias/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/field_attention_weights/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/field_attention_biases/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/content_attention_weights/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_w

Range:  [ 3 - 6 ]  Cost:  92.1436
name: "Trainer/Adam"
op: "NoOp"
input: "^Trainer/Adam/update_Unified_Vocabulary_Matrix/content_label_embedding_matrix/group_deps"
input: "^Trainer/Adam/update_Input_Embedder/field_embedding_matrix/group_deps"
input: "^Trainer/Adam/update_Encoder/rnn/lstm_cell/kernel/ApplyAdam"
input: "^Trainer/Adam/update_Encoder/rnn/lstm_cell/bias/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/field_attention_weights/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/field_attention_biases/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/content_attention_weights/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/content_attention_biases/ApplyAdam"
input: "^Trainer/Adam/update_Link_Based_Attention/trainable_weights/Link_Attention_Matrix/group_deps"
input: "^Trainer/Adam/update_Hybrid_attention/trainable_weights/zt_gate_parameter_vector/Appl

Range:  [ 0 - 3 ]  Cost:  595.7617
name: "Trainer/Adam"
op: "NoOp"
input: "^Trainer/Adam/update_Unified_Vocabulary_Matrix/content_label_embedding_matrix/group_deps"
input: "^Trainer/Adam/update_Input_Embedder/field_embedding_matrix/group_deps"
input: "^Trainer/Adam/update_Encoder/rnn/lstm_cell/kernel/ApplyAdam"
input: "^Trainer/Adam/update_Encoder/rnn/lstm_cell/bias/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/field_attention_weights/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/field_attention_biases/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/content_attention_weights/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/content_attention_biases/ApplyAdam"
input: "^Trainer/Adam/update_Link_Based_Attention/trainable_weights/Link_Attention_Matrix/group_deps"
input: "^Trainer/Adam/update_Hybrid_attention/trainable_weights/zt_gate_parameter_vector/App

Range:  [ 6 - 9 ]  Cost:  247.21162
------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------
current_epoch:  100
name: "Trainer/Adam"
op: "NoOp"
input: "^Trainer/Adam/update_Unified_Vocabulary_Matrix/content_label_embedding_matrix/group_deps"
input: "^Trainer/Adam/update_Input_Embedder/field_embedding_matrix/group_deps"
input: "^Trainer/Adam/update_Encoder/rnn/lstm_cell/kernel/ApplyAdam"
input: "^Trainer/Adam/update_Encoder/rnn/lstm_cell/bias/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/field_attention_weights/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/field_attention_biases/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/content_attention_weights/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable

Range:  [ 3 - 6 ]  Cost:  347.70148
name: "Trainer/Adam"
op: "NoOp"
input: "^Trainer/Adam/update_Unified_Vocabulary_Matrix/content_label_embedding_matrix/group_deps"
input: "^Trainer/Adam/update_Input_Embedder/field_embedding_matrix/group_deps"
input: "^Trainer/Adam/update_Encoder/rnn/lstm_cell/kernel/ApplyAdam"
input: "^Trainer/Adam/update_Encoder/rnn/lstm_cell/bias/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/field_attention_weights/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/field_attention_biases/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/content_attention_weights/ApplyAdam"
input: "^Trainer/Adam/update_Content_Based_Attention/trainable_weights/content_attention_biases/ApplyAdam"
input: "^Trainer/Adam/update_Link_Based_Attention/trainable_weights/Link_Attention_Matrix/group_deps"
input: "^Trainer/Adam/update_Hybrid_attention/trainable_weights/zt_gate_parameter_vector/Ap

In [13]:
'''
    Script for checking if the Inference computations run properly for the trained graph.
'''

from Summary_Generator.Tensorflow_Graph import order_planner_without_copynet
from Summary_Generator.Text_Preprocessing_Helpers.pickling_tools import *
from Summary_Generator.Tensorflow_Graph.utils import *
from Summary_Generator.Model import *
import numpy as np
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

In [14]:
# random_seed value for consistent debuggable behaviour
seed_value = 3

np.random.seed(seed_value) # set this seed for a device independant consistent behaviour

''' Set the constants for the script '''
# various paths of the files
data_path = "../Data" # the data path

data_files_paths = {
    "table_content": os.path.join(data_path, "train.box"),
    "nb_sentences" : os.path.join(data_path, "train.nb"),
    "train_sentences": os.path.join(data_path, "train.sent")
}

base_model_path = "Models"
plug_and_play_data_file = os.path.join(data_path, "plug_and_play.pickle")


# Set the train_percentage mark here.
train_percentage = 90

In [15]:
''' Extract and setup the data '''
# Obtain the data:
data = unPickleIt(plug_and_play_data_file)

field_encodings = data['field_encodings']
field_dict = data['field_dict']

content_encodings = data['content_encodings']

label_encodings = data['label_encodings']
content_label_dict = data['content_union_label_dict']
rev_content_label_dict = data['rev_content_union_label_dict']

# vocabulary sizes
field_vocab_size = data['field_vocab_size']
content_label_vocab_size = data['content_label_vocab_size']


X, Y = synch_random_shuffle_non_np(zip(field_encodings, content_encodings), label_encodings)

train_X, train_Y, dev_X, dev_Y = split_train_dev(X, Y, train_percentage)
train_X_field, train_X_content = zip(*train_X)
train_X_field = list(train_X_field); train_X_content = list(train_X_content)

# Free up the resources by deleting non required stuff
del X, Y, field_encodings, content_encodings, train_X

# print train_X_field, train_X_content, train_Y, dev_X, dev_Y

In [16]:
print(train_X_field, train_X_content, train_Y, dev_X, dev_Y)

[[7, 7, 7, 84, 5, 5, 5, 2, 2, 4, 4, 4, 3, 3, 3, 3, 3, 18, 9, 58, 25, 25, 25, 6, 6], [7, 2, 2, 2, 77, 78, 79, 80, 31, 31, 27, 27, 27, 81, 4, 4, 4, 3, 3, 3, 18, 9, 82, 28, 28, 28, 28, 58, 26, 26, 26, 26, 26, 26, 83, 20, 32, 55, 6, 6], [2, 2, 7, 33, 5, 29, 29, 67, 50, 68, 4, 3, 18, 9, 39, 51, 52, 6, 6], [2, 2, 7, 29, 29, 39, 4, 4, 4, 3, 3, 3, 53, 53, 69, 30, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 21, 21, 21, 21, 21, 21, 21, 21, 6, 6], [2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 18, 18, 18, 14, 14, 47, 47, 47, 7, 33, 5, 5, 5, 5, 3, 3, 3, 3, 3, 3, 3, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 22, 22, 22, 22, 22, 22, 19, 19, 19, 19, 19, 19, 19, 19, 19, 98, 48, 48, 48, 99, 100, 101, 102, 49, 49, 49, 103, 104, 105, 6, 6, 6], [2, 2, 7, 7, 7, 7, 7, 7, 7, 33, 5, 5, 5, 5, 5, 5, 30, 30, 44, 44, 44, 59, 85, 86, 4, 4, 